# စာသားအမျိုးအစားသတ်မှတ်ခြင်း အလုပ်

ဒီအခန်းမှာတော့ **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)** dataset ကိုအခြေခံပြီး ရိုးရှင်းတဲ့ စာသားအမျိုးအစားသတ်မှတ်ခြင်း အလုပ်ကို စတင်လုပ်ဆောင်ပါမယ်။ ဤအလုပ်မှာ သတင်းခေါင်းစဉ်များကို အမျိုးအစား ၄ မျိုးဖြစ်တဲ့ World, Sports, Business, Sci/Tech တို့ထဲက တစ်ခုအဖြစ် သတ်မှတ်ပေးပါမယ်။

## Dataset အကြောင်း

Dataset ကို load လုပ်ဖို့အတွက် **[TensorFlow Datasets](https://www.tensorflow.org/datasets)** API ကို အသုံးပြုပါမယ်။


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

အခုကျွန်တော်တို့ `dataset['train']` နဲ့ `dataset['test']` ကိုအသုံးပြုပြီး training portion နဲ့ test portion ကို dataset မှာ access လုပ်နိုင်ပါပြီ:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


အချက်အလက်စုစည်းမှုမှ ပထမဆုံး ၁၀ ခုသော သတင်းခေါင်းစဉ်အသစ်များကို ပုံနှိပ်ကြမယ်:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## စာသားကို ဂဏန်းအဖြစ်ပြောင်းခြင်း

အခုတော့ စာသားတွေကို **ဂဏန်း** အဖြစ်ပြောင်းပြီး tensor အနေနဲ့ ကိုယ်စားပြုနိုင်အောင် ပြုလုပ်ရပါမယ်။ စကားလုံးအဆင့်ကိုယ်စားပြုမှုလိုချင်ရင် အောက်ပါအချက်နှစ်ခုကို ပြုလုပ်ရပါမယ်-

* **tokenizer** ကိုသုံးပြီး စာသားကို **tokens** အဖြစ်ခွဲခြားပါ။
* အဲ့ဒီ tokens တွေကို **vocabulary** တစ်ခုတည်ဆောက်ပါ။

### Vocabulary အရွယ်အစားကို ကန့်သတ်ခြင်း

AG News dataset ဥပမာမှာတော့ vocabulary size က အတော်လေးကြီးပါတယ်၊ စကားလုံး 100,000 ကျော်ပါဝင်ပါတယ်။ အထူးသဖြင့် စာသားမှာ ရှားရှားပါးပါးပေါ်လာတဲ့ စကားလုံးတွေကို မလိုအပ်ပါဘူး — အဲ့ဒီစကားလုံးတွေဟာ စာကြောင်းအနည်းငယ်မှာပဲ ပါဝင်ပြီး၊ model က အဲ့ဒီစကားလုံးတွေကနေ သင်ယူနိုင်မှာ မဟုတ်ပါဘူး။ ဒါကြောင့် vocabulary size ကို သေးငယ်တဲ့အရေအတွက်တစ်ခုအထိ ကန့်သတ်ဖို့ make sense ဖြစ်ပါတယ်၊ အဲ့ဒီအတွက် vectorizer constructor ကို argument ဖြတ်ပေးရပါမယ်။

အဲ့ဒီအဆင့်နှစ်ခုကို **TextVectorization** layer ကိုသုံးပြီး လုပ်ဆောင်နိုင်ပါတယ်။ အရင်ဆုံး vectorizer object ကို instantiate လုပ်ပြီး၊ `adapt` method ကို ခေါ်သုံးကာ စာသားအားလုံးကို ဖြတ်သွားပြီး vocabulary တစ်ခုတည်ဆောက်ပါ။


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **မှတ်ချက်** ကျွန်တော်တို့သည် စာလုံးစုစည်းမှုတစ်ခုကို တည်ဆောက်ရန် အချက်အလက်စုစည်းမှု၏ အစိတ်အပိုင်းတစ်ခုကိုသာ အသုံးပြုနေပါသည်။ ဒါကို လုပ်ခြင်းဖြင့် အကောင်အထည်ဖော်မှုအချိန်ကို မြန်ဆန်စေပြီး သင့်ကို စောင့်နေစရာမလိုအောင် လုပ်ဆောင်ပါသည်။ သို့သော် အချက်အလက်စုစည်းမှု၏ စာလုံးအချို့သည် စာလုံးစုစည်းမှုထဲတွင် မပါဝင်နိုင်ဘဲ လေ့ကျင့်မှုအတွင်း မျက်မမြင်ဖြစ်နိုင်သည်ဆိုသော အန္တရာယ်ကို ကျွန်တော်တို့ ခံယူထားပါသည်။ ထို့ကြောင့် `adapt` လုပ်ဆောင်မှုအတွင်း စာလုံးစုစည်းမှုအရွယ်အစားအားလုံးကို အသုံးပြုပြီး အချက်အလက်စုစည်းမှုအားလုံးကို ဖြတ်သန်းခြင်းဖြင့် နောက်ဆုံးရလဒ်တိကျမှုကို တိုးမြှင့်နိုင်မည်ဖြစ်သော်လည်း အလွန်အမင်း မတိုးတက်နိုင်ပါ။

အခု ကျွန်တော်တို့ စာလုံးစုစည်းမှုကို တကယ်ရောက်ရှိနိုင်ပါပြီ:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


ဗက်တိုရာဇာကို အသုံးပြု၍ မည်သည့်စာသားကိုမဆို အလွယ်တကူ နံပါတ်များအဖြစ် ကုဒ်ပြုလုပ်နိုင်ပါသည်။


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## စကားလုံးအထုပ် (Bag-of-words) စာသားကိုယ်စားပြုမှု

စကားလုံးများသည် အဓိပ္ပါယ်ကို ကိုယ်စားပြုနိုင်သောကြောင့်၊ တစ်ခါတစ်ရံ စာသားတစ်ခု၏ အဓိပ္ပါယ်ကို စာကြောင်းအတွင်း စကားလုံးများ၏ အစီအစဉ်ကို မကြည့်ဘဲ၊ တစ်ခုချင်းစီ စကားလုံးများကိုသာ ကြည့်ခြင်းဖြင့် သိနိုင်ပါသည်။ ဥပမာအားဖြင့် သတင်းများကို အမျိုးအစားခွဲခြားရာတွင် *ရာသီဥတု* နှင့် *နှင်း* ကဲ့သို့သော စကားလုံးများသည် *ရာသီဥတုခန့်မှန်းချက်* ကို ဖော်ပြနိုင်ပြီး၊ *အหุ้น* နှင့် *ဒေါ်လာ* ကဲ့သို့သော စကားလုံးများသည် *ဘဏ္ဍာရေးသတင်း* ကို ဖော်ပြနိုင်ပါသည်။

**စကားလုံးအထုပ်** (BoW) ဗက်တာကိုယ်စားပြုမှုသည် နားလည်ရန် အလွယ်ဆုံးသော ရိုးရာဗက်တာကိုယ်စားပြုမှုဖြစ်သည်။ စကားလုံးတစ်ခုချင်းစီကို ဗက်တာအညွှန်းနှင့် ချိတ်ဆက်ထားပြီး၊ ဗက်တာအခန်းကဏ္ဍတစ်ခုတွင် သတ်မှတ်ထားသော စာရွက်စာတမ်းအတွင်း စကားလုံးတစ်ခုချင်းစီ၏ ဖြစ်ပေါ်မှုအရေအတွက်ကို ပါဝင်ထားသည်။

![စကားလုံးအထုပ်ဗက်တာကိုယ်စားပြုမှုကို မှတ်ဉာဏ်တွင် ဘယ်လိုကိုယ်စားပြုထားသည်ကို ဖော်ပြထားသော ပုံ။](../../../../../translated_images/my/bag-of-words-example.606fc1738f1d7ba9.webp)

> **Note**: BoW ကို စာသားအတွင်း စကားလုံးတစ်ခုချင်းစီအတွက် တစ်ခုချင်းစီ *one-hot-encoded* ဗက်တာများ၏ စုစုပေါင်းအဖြစ်လည်း စဉ်းစားနိုင်ပါသည်။

အောက်တွင် Scikit Learn python library ကို အသုံးပြု၍ စကားလုံးအထုပ်ကိုယ်စားပြုမှုကို ဖန်တီးပုံ၏ ဥပမာကို ဖော်ပြထားပါသည်။


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

ကျွန်ုပ်တို့အပေါ်တွင်သတ်မှတ်ထားသော Keras vectorizer ကိုလည်းအသုံးပြုနိုင်ပြီး၊ စကားလုံးနံပါတ်တစ်ခုချင်းစီကို one-hot encoding အဖြစ်ပြောင်းလဲပြီး၊ အဲဒီ vector တွေကိုလုံးလုံးပေါင်းထည့်နိုင်ပါသည်။


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **မှတ်ချက်**: အရင်ဥပမာနဲ့ရလဒ်ကွာခြားနေတယ်ဆိုတာကို သင်အံ့ဩနိုင်ပါတယ်။ အကြောင်းက Keras ဥပမာမှာ vector ရဲ့အရှည်ဟာ vocabulary size နဲ့ကိုက်ညီပြီး၊ အဲဒီ vocabulary ကို AG News dataset အပြည့်အစုံကနေတည်ဆောက်ထားတာဖြစ်ပါတယ်။ ဒါပေမယ့် Scikit Learn ဥပမာမှာတော့ vocabulary ကို sample text ကနေ ချက်ချင်းတည်ဆောက်ထားတာဖြစ်ပါတယ်။


## BoW ခွဲခြားစနစ်ကို လေ့ကျင့်ခြင်း

အခုတော့ ကျွန်တော်တို့ စာသားကို bag-of-words ကိုယ်စားပြုမှုအဖြစ် တည်ဆောက်နည်းကို သင်ယူပြီးဖြစ်သောကြောင့်၊ ဒါကို အသုံးပြုတဲ့ ခွဲခြားစနစ်တစ်ခုကို လေ့ကျင့်ကြမယ်။ ပထမဦးစွာ ကျွန်တော်တို့ရဲ့ ဒေတာစုစည်းမှုကို bag-of-words ကိုယ်စားပြုမှုအဖြစ် ပြောင်းလဲဖို့ လိုအပ်ပါတယ်။ ဒါကို `map` function ကို အောက်ပါနည်းလမ်းဖြင့် အသုံးပြုခြင်းဖြင့် ပြုလုပ်နိုင်ပါတယ် - 


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

အခုတော့ linear layer တစ်ခုပါဝင်တဲ့ ရိုးရှင်းတဲ့ classifier neural network ကို သတ်မှတ်ကြမယ်။ Input size က `vocab_size` ဖြစ်ပြီး၊ output size က class အရေအတွက် (၄) ကို ကိုယ်စားပြုပါတယ်။ Classification task ကို ဖြေရှင်းနေတာဖြစ်လို့၊ နောက်ဆုံး activation function က **softmax** ဖြစ်ပါတယ်။


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


ကျွန်တော်တို့မှာ အတန်း ၄ ခုရှိတဲ့အတွက် ၈၀% အထက်ရှိတဲ့ တိကျမှုဟာ ရလဒ်ကောင်းတစ်ခုဖြစ်ပါတယ်။

## တစ်ခုတည်းသော network အဖြစ် classifier ကို လေ့ကျင့်ခြင်း

Vectorizer ကလည်း Keras layer တစ်ခုဖြစ်တဲ့အတွက်၊ network တစ်ခုအဖြစ် သတ်မှတ်ပြီး၊ အဆုံးအထိ လေ့ကျင့်နိုင်ပါတယ်။ ဒီနည်းလမ်းနဲ့ dataset ကို `map` အသုံးပြုပြီး vectorize လုပ်စရာမလိုတော့ပါဘူး၊ network ရဲ့ input ကို အစစ်အမှန် dataset ကိုပဲ ဖြတ်သွားနိုင်ပါတယ်။

> **Note**: Dataset ကို dictionary (ဥပမာ `title`, `description` နဲ့ `label`) ကနေ tuple အဖြစ် ပြောင်းဖို့ map တွေကို အသုံးပြုရဦးမှာဖြစ်ပါတယ်။ သို့သော် disk ကနေ data ကို load လုပ်တဲ့အခါမှာတော့ လိုအပ်တဲ့ ဖွဲ့စည်းမှုနဲ့ dataset ကို အစဉ်အတိုင်း တည်ဆောက်နိုင်ပါတယ်။


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigrams, trigrams နှင့် n-grams

Bag-of-words နည်းလမ်း၏ အားနည်းချက်တစ်ခုမှာ စကားလုံးတစ်ချို့သည် စကားလုံးများပေါင်းစပ်မှုဖြစ်ပြီး၊ ဥပမာအားဖြင့် 'hot dog' ဆိုသော စကားလုံးသည် 'hot' နှင့် 'dog' ဆိုသော စကားလုံးများ၏ အခြားအကြောင်းအရာများတွင် အဓိပ္ပါယ်ကွဲပြားမှုရှိသည်။ 'hot' နှင့် 'dog' စကားလုံးများကို အမြဲတမ်းတူညီသော vectors ဖြင့် ကိုယ်စားပြုပါက၊ ၎င်းသည် မော်ဒယ်ကို ရှုပ်ထွေးစေနိုင်သည်။

ဤပြဿနာကို ဖြေရှင်းရန်၊ **n-gram ကိုယ်စားပြုမှုများ** ကို အချို့သော စာရွက်စာတမ်းများကို အမျိုးအစားခွဲခြားရာတွင် အသုံးပြုလေ့ရှိပြီး၊ စကားလုံးတစ်လုံး၊ စကားလုံးနှစ်လုံး (bi-word) သို့မဟုတ် စကားလုံးသုံးလုံး (tri-word) တစ်ခုချင်းစီ၏ frequency သည် classifier များကို လေ့ကျင့်ရန် အသုံးဝင်သော feature ဖြစ်သည်။ ဥပမာအားဖြင့် bigram ကိုယ်စားပြုမှုတွင်၊ မူရင်းစကားလုံးများအပြင် စကားလုံးအတွဲများအားလုံးကို vocabulary ထဲသို့ ထည့်သွင်းပါမည်။

အောက်တွင် Scikit Learn ကို အသုံးပြု၍ bigram bag of word ကိုယ်စားပြုမှုကို ဖန်တီးနည်း၏ ဥပမာကို ဖော်ပြထားသည်-


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

n-gram နည်းလမ်းရဲ့ အဓိကအားနည်းချက်က vocabulary အရွယ်အစားက အလွန်မြန်မြန်ဆန်ဆန် ကြီးထွားလာတတ်တာပဲ ဖြစ်ပါတယ်။ အကောင်အထည်ဖော်ရာမှာတော့ n-gram ကို အသုံးပြုတဲ့အခါ dimensionality reduction နည်းလမ်းတစ်ခုဖြစ်တဲ့ *embeddings* နဲ့ပေါင်းစပ်ဖို့ လိုအပ်ပါတယ်၊ ဒီအကြောင်းကို နောက်အခန်းမှာ ဆွေးနွေးပါမယ်။

**AG News** dataset မှာ n-gram ကို အသုံးပြုဖို့ `ngrams` parameter ကို `TextVectorization` constructor ထဲမှာ ဖြတ်သွင်းရပါမယ်။ bigram vocabulary ရဲ့ အရှည်က **အလွန်ကြီးမား**ပြီး၊ ကျွန်တော်တို့ရဲ့ အခြေအနေမှာတော့ 1.3 million tokens ထက်ပိုပါတယ်! ဒါကြောင့် bigram tokens ကိုလည်း သင့်တော်တဲ့ အရေအတွက်တစ်ခုနဲ့ ကန့်သတ်ဖို့ make sense ဖြစ်ပါတယ်။

classifier ကို training လုပ်ဖို့ အပေါ်မှာ အသုံးပြုခဲ့တဲ့ code ကိုပဲ အသုံးပြုနိုင်ပါတယ်၊ ဒါပေမယ့် memory ကို အလွန်မထိရောက်စွာ အသုံးပြုမိနိုင်ပါဘူး။ နောက်အခန်းမှာတော့ embeddings ကို အသုံးပြုပြီး bigram classifier ကို training လုပ်ပါမယ်။ အခုအချိန်မှာတော့ ဒီ notebook ထဲမှာ bigram classifier training ကို စမ်းသပ်ပြီး အတိအကျမှု (accuracy) ပိုမြင့်တင်နိုင်မလား စမ်းကြည့်နိုင်ပါတယ်။


## BoW ဗက်တာများကို အလိုအလျောက်တွက်ချက်ခြင်း

အထက်ပါ ဥပမာတွင် BoW ဗက်တာများကို တစ်ခုချင်းစီသော စကားလုံးများ၏ one-hot encodings များကို ပေါင်းပြီး လက်ဖြင့်တွက်ချက်ခဲ့ပါသည်။ သို့သော် TensorFlow ၏ နောက်ဆုံးဗားရှင်းတွင် `output_mode='count` parameter ကို vectorizer constructor သို့ ဖြတ်သွားခြင်းဖြင့် BoW ဗက်တာများကို အလိုအလျောက်တွက်ချက်နိုင်ပါသည်။ ဤနည်းလမ်းသည် မော်ဒယ်ကို သတ်မှတ်ခြင်းနှင့် လေ့ကျင့်ခြင်းကို အလွန်လွယ်ကူစေပါသည်။


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## စကားလုံးအကြိမ်ရေ - စာရွက်စာတမ်းအကြိမ်ရေကို ဆန့်ကျင်သော အကြိမ်ရေ (TF-IDF)

BoW ကိုယ်စားပြုမှုတွင် စကားလုံးများ၏ ဖြစ်ပေါ်မှုကို စကားလုံး၏ အမျိုးအစားမရွေးဘဲ တူညီသောနည်းလမ်းဖြင့် အလေးပေးသည်။ သို့သော် *a* နှင့် *in* ကဲ့သို့ မကြာခဏ တွေ့ရသော စကားလုံးများသည် အထူးသီးသန့် စကားလုံးများထက် အမျိုးအစားခွဲခြားမှုအတွက် အရေးပါမှုနည်းပါးသည်ဟု သိသာသည်။ NLP အလုပ်များအများစုတွင် စကားလုံးတချို့သည် အခြားစကားလုံးများထက် ပိုမိုသက်ဆိုင်မှုရှိသည်။

**TF-IDF** သည် **term frequency - inverse document frequency** ကို ဆိုလိုသည်။ ၎င်းသည် bag-of-words ၏ အမျိုးအစားတစ်ခုဖြစ်ပြီး စကားလုံးတစ်ခုသည် စာရွက်စာတမ်းတွင် ပါဝင်မှုကို binary 0/1 တန်ဖိုးဖြင့် ဖော်ပြခြင်းမဟုတ်ဘဲ စကားလုံး၏ corpus တွင် ဖြစ်ပေါ်မှုအကြိမ်ရေနှင့် ဆက်စပ်သော floating-point တန်ဖိုးကို အသုံးပြုသည်။

ပိုမိုတိကျစွာဆိုရမည်ဆိုလျှင် စကားလုံး $i$ ၏ စာရွက်စာတမ်း $j$ တွင် အလေးပေးမှု $w_{ij}$ ကို အောက်ပါအတိုင်း သတ်မှတ်သည်။
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
အဲဒီမှာ
* $tf_{ij}$ သည် $i$ ကို $j$ တွင် တွေ့ရသော အကြိမ်ရေဖြစ်သည်၊ ဒါဟာ ကျွန်တော်တို့ အရင်က တွေ့ခဲ့တဲ့ BoW တန်ဖိုးဖြစ်သည်
* $N$ သည် စုစုပေါင်း စာရွက်စာတမ်းအရေအတွက်ဖြစ်သည်
* $df_i$ သည် စကားလုံး $i$ ကို စုစုပေါင်း စာရွက်စာတမ်းများတွင် ပါဝင်သော စာရွက်စာတမ်းအရေအတွက်ဖြစ်သည်

TF-IDF တန်ဖိုး $w_{ij}$ သည် စကားလုံးတစ်ခုသည် စာရွက်စာတမ်းတွင် ပါဝင်သော အကြိမ်ရေအတိုင်း တိုးမြှင့်ပြီး corpus တွင် စကားလုံးပါဝင်သော စာရွက်စာတမ်းအရေအတွက်ကို အလေးပေးမှုဖြင့် လျှော့ချသည်။ ၎င်းသည် စကားလုံးတစ်ချို့သည် အခြားစကားလုံးများထက် မကြာခဏ တွေ့ရသည်ဆိုသော အချက်ကို ပြင်ဆင်ရန် အထောက်အကူပြုသည်။ ဥပမာအားဖြင့် စကားလုံးတစ်ခုသည် စုစုပေါင်း စာရွက်စာတမ်းများ *အားလုံး* တွင် ပါဝင်လျှင် $df_i=N$ ဖြစ်ပြီး $w_{ij}=0$ ဖြစ်သည်။ ထို့ကြောင့် စကားလုံးများကို လုံးဝ မထည့်သွင်းစဉ်းစားတော့ပါ။

Scikit Learn ကို အသုံးပြု၍ TF-IDF vectorization ကို လွယ်ကူစွာ ဖန်တီးနိုင်သည်။


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

Keras တွင် `TextVectorization` layer သည် `output_mode='tf-idf'` parameter ကို ဖြတ်သွားခြင်းဖြင့် TF-IDF frequency များကို အလိုအလျောက်တွက်ချက်နိုင်သည်။ TF-IDF ကို အသုံးပြုခြင်းက တိကျမှုကို တိုးမြှင့်ပေးနိုင်မည်လားဆိုတာကို ကြည့်ရန် အထက်တွင် အသုံးပြုခဲ့သော code ကို ထပ်မံလုပ်ဆောင်ကြည့်ပါ။


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## အနှစ်ချုပ်

TF-IDF ကိုယ်စားပြုမှုများသည် စကားလုံးများအတွက် ကြိမ်နှုန်းအလေးချိန်များကို ပေးနိုင်သော်လည်း အဓိပ္ပါယ် သို့မဟုတ် အစီအစဉ်ကို ကိုယ်စားပြုနိုင်ခြင်း မရှိပါ။ 1935 ခုနှစ်တွင် နာမည်ကြီး ဘာသာဗေဒပညာရှင် J. R. Firth က ပြောခဲ့သလို၊ "စကားလုံးတစ်လုံး၏ အပြည့်အစုံသော အဓိပ္ပါယ်သည် အမြဲတမ်း အခြေအနေအရသာသာ ရှိနိုင်ပြီး အခြေအနေမှ ကွဲလွဲသော အဓိပ္ပါယ်ကို လေ့လာခြင်းသည် အလေးထားစဉ်းစားရန် မဖြစ်နိုင်ပါ။" ကျွန်ုပ်တို့သည် သင်တန်း၏ နောက်ပိုင်းတွင် ဘာသာစကား မော်ဒယ်တစ်ခုကို အသုံးပြု၍ စာသားမှ အခြေအနေဆိုင်ရာ အချက်အလက်များကို ဖမ်းယူနည်းကို လေ့လာသွားပါမည်။



---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေပါသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါရှိနိုင်သည်ကို သတိပြုပါ။ မူရင်းဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာရှိသော ရင်းမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူက ဘာသာပြန်မှု ဝန်ဆောင်မှုကို အသုံးပြုရန် အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအလွတ်များ သို့မဟုတ် အနားလွဲမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
